## Examining data
Import all necessary libraries and save data from files to variables.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as st
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

try:
    calls = pd.read_csv('calls.csv')  # Local path 
    internet = pd.read_csv('internet.csv')
    messages = pd.read_csv('messages.csv')
    tariffs = pd.read_csv('tariffs.csv')
    users = pd.read_csv('users.csv')
except:
    calls = pd.read_csv('/datasets/calls.csv')  # Server path
    internet = pd.read_csv('/datasets/internet.csv')
    messages = pd.read_csv('/datasets/messages.csv')
    tariffs = pd.read_csv('/datasets/tariffs.csv')
    users = pd.read_csv('/datasets/users.csv')
  


calls.info()
internet.info()
messages.info()
tariffs.info()
users.info()

,id,call_date,duration,user_id
0,1000_0,2018-07-25,0.00,1000
1,1000_1,2018-08-17,0.00,1000
2,1000_2,2018-06-11,2.85,1000
3,1000_3,2018-09-21,13.80,1000
4,1000_4,2018-12-15,5.18,1000
...,...,...,...,...
202602,1499_215,2018-12-26,0.76,1499
202603,1499_216,2018-10-18,18.83,1499
202604,1499_217,2018-11-10,10.81,1499
202605,1499_218,2018-10-06,4.27,1499


Following data type issues were found in the "calls" table:
- call_date object type, we will convert it into a date type, for the convenience of further use

Following data type issues were found in the "internet" table:
- the column Unnamed: 0 completely duplicates the indexes, we will most likely get rid of it
- session_date object type, convert to date type, for ease of further use

Following data type issues were found in the "messages" table:
- message_date object type, we will convert it into date type, for the convenience of further use

No data type issues found in "tariffs" table

Following data type issues were found in the "users" table:
- churn_date type object and reg_date type object, we will convert into date type, for convenience of further use
- churn_date filled in only 38 values, but this indicator in this study does not bother us, so let's leave it as it is

In the "calls", "internet" tables, there are a small number of 0 values in the duration, mb_used columns, respectively. Calls with zero duration are missed calls, so we won't delete them. As for the zero values for mb_used, this data will not affect the study, since we will use the total value of the internet spending per month, so we can just leave them.

Also check the main tables for duplicates:

In [ ]:
print(calls.duplicated().sum(), internet.duplicated().sum(), messages.duplicated().sum(), users.duplicated().sum())

There is no duplicates in tables.

## Data preprocessing

In [ ]:
#Convert all formats as described above
calls['call_date'] = pd.to_datetime(calls['call_date'], format='%Y-%m-%d')
internet['session_date'] = pd.to_datetime(internet['session_date'], format='%Y-%m-%d')
messages['message_date'] = pd.to_datetime(messages['message_date'], format='%Y-%m-%d')
users['churn_date'] = pd.to_datetime(users['churn_date'], format='%Y-%m-%d')
users['reg_date'] = pd.to_datetime(users['churn_date'], format='%Y-%m-%d')

To test hypotheses, some indicators need to be calculated. Let's calculate for each user:
- number of calls made and minutes of conversation spent per month;
- number of messages sent per month;
- amount of Internet traffic used per months;
- monthly revenue per each user

Let's define a month from data for message sent, Internet traffic spent and calls made, for this we create a month column in each table.

In [ ]:
calls['month'] = calls['call_date'].dt.to_period('M')
internet['month'] = internet['session_date'].dt.to_period('M')
messages['month'] = messages['message_date'].dt.to_period('M')

Now let's create pivot tables in which we calculate the total number of minutes, messages and gigabytes spent for each user in each month.

In [ ]:
calls_pivot = calls.pivot_table(index=['user_id', 'month'], values='duration', aggfunc='sum')
calls_pivot = calls_pivot.reset_index(drop=False)
calls_pivot.columns = ['user_id', 'month', 'calls_duration']

internet_pivot = internet.pivot_table(index=['user_id', 'month'], values='mb_used', aggfunc='sum')
internet_pivot = internet_pivot.reset_index(drop=False)
internet_pivot.columns = ['user_id', 'month', 'mb_used_month']

messages_pivot = messages.pivot_table(index=['user_id', 'month'], values='id', aggfunc='count')
messages_pivot = messages_pivot.reset_index(drop=False)
messages_pivot.columns = ['user_id', 'month', 'message_count']

Next, we will create a common pivot table by combining the data from these tables, additionally adding columns with the name of the tariff and the user's city to it and see what happens.

In [ ]:
mobile_data = calls_pivot.merge(internet_pivot, on = ['user_id', 'month'], how='outer')
mobile_data = mobile_data.merge(messages_pivot, on = ['user_id', 'month'], how='outer')

tarif = users.loc[:, ['user_id', 'tariff']]
city = users.loc[:, ['user_id', 'city']]

In [ ]:
mobile_data = mobile_data.merge(tarif, on = 'user_id', how='outer')
mobile_data = mobile_data.merge(city, on = 'user_id', how='outer')
display(mobile_data.head())

In [ ]:
mobile_data.shape

In [ ]:
mobile_data.user_id.nunique()

Next, we will calculate the revenue for each row of the mobile_data pivot table, for this we subtract the free limit from the total number of calls, messages and Internet traffic; multiply the remainder by the value from the tariff plan and add the subscription fee, in accordance with the tariff plan. We also take into account that Internet traffic excesses are paid per gigabyte, so we will translate the result of the excess from megabytes into gigabytes and only then multiply by the tariff.

In [ ]:
display(tariffs)

In [ ]:
def revenue_count(row):
    user = row['user_id']
    calls_duration = row['calls_duration']
    mb_used_month = row['mb_used_month']
    message_count = row['message_count']
    tariff = row['tariff']
    revenue = 0

    if tariff == 'smart':
        messages_included = 50   
        mb_per_month_included = 15360
        minutes_included = 500
        revenue = 550
        rub_per_gb = 200
        rub_per_message = 3
        rub_per_minute = 3
    
    if tariff == 'ultra':
        messages_included = 1000
        mb_per_month_included = 30720
        minutes_included = 3000
        revenue = 1950
        rub_per_gb = 150
        rub_per_message = 1
        rub_per_minute = 1    
    if calls_duration > minutes_included:
        revenue = revenue + ((calls_duration - minutes_included)*rub_per_minute)
    if mb_used_month > mb_per_month_included:
        revenue = revenue + ((np.ceil((mb_used_month - mb_per_month_included)/1024))*rub_per_gb)
    if message_count > messages_included:
        revenue = revenue + ((message_count - messages_included)*rub_per_message)
    return(revenue)

mobile_data['revenue'] = mobile_data.apply(revenue_count, axis=1)
mobile_data.head(10)

In [ ]:
mobile_data['revenue'].describe()

## Data Analysis 

Let's divide the pivot table into two based on the tariff in order to analyze the behavior of users of different tariffs relative to each other. We will also calculate the average number, variance and standard deviation of the number of minutes of conversation, the number of messages and the volume of Internet traffic required by users of each tariff per month

In [ ]:
good_data_smart = mobile_data.query('tariff == "smart"')
good_data_ultra = mobile_data.query('tariff == "ultra"')

In [ ]:
#таблица.pivot_table(index= тариф  values=  выручка, aggfunc= ['mean', 'var','std'])
mobile_data_pivot = mobile_data.pivot_table(index='tariff', values='revenue', aggfunc=['mean', 'var','std', 'count'])
display(mobile_data_pivot)
mobile_data_pivot = mobile_data.pivot_table(index='tariff', values='message_count', aggfunc=['mean', 'var','std'])
display(mobile_data_pivot)
mobile_data_pivot = mobile_data.pivot_table(index='tariff', values='mb_used_month', aggfunc=['mean', 'var','std'])
display(mobile_data_pivot)
mobile_data_pivot = mobile_data.pivot_table(index='tariff', values='calls_duration', aggfunc=['mean', 'var','std'])
display(mobile_data_pivot)

According to the calculations above, I would like to note the following:
- the average revenue for the ultra tariff greatly exceeds the revenue for the smart tariff, but at the same time, the standard deviation and variance for the smart tariff are higher, which indicates a high variability of the random variable relative to its mathematical expectation.
- there is more data on the smart tariff than on the ultra tariff, which is logical, since the ultra tariff is more expensive and, accordingly, less popular.
- users of the ultra tariff on average write SMS more often than users of the smart tariff
- users of the ultra tariff on average use mobile Internet more than users of the smart tariff
- users of the ultra tariff on average spend more time talking on the phone than users of the smart tariff

Now let's build histograms of the distribution of these data in order to visually assess the nature of the behavior of users of different tariffs.

In [ ]:
ax = good_data_smart.plot(kind='hist', y='calls_duration', bins=50, alpha=0.7, label='calls duration smart', grid=True, legend=True, figsize=(12, 7))
good_data_ultra.plot(kind='hist', y='calls_duration', bins=50, alpha=0.7, label='calls duration ultra', ax=ax, grid=True, legend=True, figsize=(12, 7))
plt.title('Total calls duration per month')
plt.show()

The histogram shows that the peak of the distribution of talk time at the smart tariff is observed in the region of 500 minutes, and after this mark it drops sharply down, perhaps since the free number of minutes is only 500, those who often talk keep track of the rest of the minutes so as not to switch to paid part, or even if the limit is exhausted, then do not spend too much on exceeding, because the cost of a minute over the limit is high. According to the ultra tariff graph, this is not observed, since the limit of 3000 minutes allows you to relax and not keep track of how many minutes have already been spent, so the graph has no obvious peaks. And, as we see from the data presented, this limit has not been exceeded by anyone from the sample, that is, the probability of this is low.

In [ ]:
ax = good_data_smart.plot(kind='hist', y='mb_used_month', bins=100, alpha=0.7, label='mb used smart', grid=True, legend=True, figsize=(12, 7))
good_data_ultra.plot(kind='hist', y='mb_used_month', bins=100, alpha=0.7, label='mb used ultra', ax=ax, grid=True, legend=True, figsize=(12, 7))
plt.title('Total internet usage per month')
plt.show()

According to the Internet consumption graph at the smart tariff, there is also a clear peak in the area of the traffic limit, and this can also be described by the desire of users not to go beyond the limit included in the tariff, after 15000 there is no longer such a sharp decrease in the histogram, it looks more like a normal distribution curve, to me this seems to be due to the fact that now access to the Internet is an important part of life and users use Internet when necessary and are probably more willing to overpay for it than for calls.

There are no clear peaks in the ultra tariff, since 30 gigabytes is a rather large amount of traffic and you need to try hard to use it up, there is still a small step on the threshold of the schedule limit, that is, a small percentage of people approaching the mark may begin to save traffic in order to do not overpay, but at the same time, the step is not very pronounced and we can say that users who spend more Internet than are provided within the framework of the tariff are an absolute minority and the bulk of users fit into the free limit.

In [ ]:
ax = good_data_smart.plot(kind='hist', y='message_count', bins=50, alpha=0.7, label='messages smart', grid=True, legend=True, figsize=(12, 7))
good_data_ultra.plot(kind='hist', y='message_count', bins=50, alpha=0.7, label='messages ultra', ax=ax, grid=True, legend=True, figsize=(12, 7))
plt.title('Message amount per month')
plt.show()

"Smart" tariff shows more monotonous picture up to the free limit with a small peak, then there is a sharp decline, I think this is due to the high cost of 1 SMS after spending the limit. In the ultra tariff, the graph has no pronounced peaks, due to the fact that the limit is 1000 sms and this is a rather large value, and the graph shows that no one from the sample even came close to using the limit. SMS is not the most popular means of communication today, and a fairly large percentage of users do not use it at all, which is generally logical in the era of mobile Internet and instant messengers.

In [ ]:
ax = good_data_smart.plot(kind='hist', y='revenue', bins=70, alpha=0.7, label='revenue smart', grid=True, legend=True, figsize=(12, 7))
good_data_ultra.plot(kind='hist', y='revenue', bins=70, alpha=0.7, label='revenue ultra', ax=ax, grid=True, legend=True, figsize=(12, 7))
plt.title('Revenue per month')
plt.show()

According to the histograms, it can be seen that the vast majority of users of the ultra tariff fit into the tariff limits and pay only the cost of the tariff. In the smart tariff, it is more difficult to meet the limits, since they are not so high, so you can see a long and thick tail of payments with tariff excesses. Let's check the proportion of those who fit into the smart and ultra tariff and those who do not fit:

In [ ]:
print('Do not fit to tariff smart', (len(good_data_smart.loc[good_data_smart['revenue'] > 550]))/ (len(good_data_smart)))
print('Do not fit to tariff ultra', (len(good_data_ultra.loc[good_data_ultra['revenue'] > 1950]))/ (len(good_data_ultra)))

In 73% of cases, users do not fit into the smart tariff and only in 13% into the ultra tariff

## Testing hypotheses

Let us formulate a null hypothesis for verification - the average revenue of users of the Ultra and Smart tariffs are equal, which means that the alternative hypothesis will be - the average revenue of users of the Ultra and Smart tariffs are different. This alternative hypothesis is two-sided, since we are interested in the deviation in both directions. Since our samples are of different sizes and the variances of the populations are different, as we already calculated above, we will specify the equal_var parameter as False, we will concider the alpha value of 5%.

In [ ]:
def analysis(array_1, array_2):
    alpha = 0.05
    results = st.stats.ttest_ind(array_1, array_2, equal_var = False)
    print('p-value: ', results.pvalue)
    if results.pvalue < alpha: 
        print ("Deny zero hypothesis")
    else:
        print ("Cannot deny zero hypothesis")

In [ ]:
# Null hypothesis: the average revenue for smart and ultra tariffs are equal.
# Alternative hypothesis: the average revenue for smart and ultra tariffs differ

display(mobile_data_pivot)

# The variance of the two samples differ by more than 4 times, quite a big difference, as well as the size
# samples are very different, so we set the equal_var parameter to False

analysis(good_data_smart['revenue'], good_data_ultra['revenue'])

With the help of the test, the null hypothesis was rejected, and the alternative one was confirmed, namely, that the average revenue for smart and ultra tariffs differ. This was presumably clear, and by calculating the average value of the sample, they have too much difference, now we know this for sure.


Now let's test another hypothesis that the average revenue for MSC users differs from the revenue for users from regions. This will be our null hypothesis, the alternative one says: the average revenue for MSC users is equal to the revenue for users from regions.

In order to test this hypothesis, it is necessary to prepare datasets for testing, that is, filter the data by users from Moscow and the regions. Let's place them in different variables. Next, we compare the variances and sizes of the resulting samples, as well as their average values.

In [ ]:
array_moscow = mobile_data.query('city == "Москва"')
array_region = mobile_data.query('city != "Москва"')

print(array_region['revenue'].mean(), array_moscow['revenue'].mean())
print(np.var(array_region['revenue']), np.var(array_moscow['revenue']))
print(len(array_region['revenue']), len(array_moscow['revenue']))


As in the first case, despite the fact that the variance differs little, our samples are of different sizes, therefore, just as in the first case, we will specify the equal_var parameter as False, and we will equate the threshold value alpha to 5%

In [ ]:
# Null hypothesis: the average revenue in Moscow and regions are equal
# Alternative hypothesis: the average revenue in Moscow and regions differ

analysis(array_moscow['revenue'], array_region['revenue'])

As can be seen from the average value and the test results, the null hypothesis is correct, namely, that the average revenue in Moscow and the regions does not differ.


## General conclusion:

- As can be seen from the histograms, the vast majority of Ultra tariff users fit into the tariff limits, and only 12% pay extra for excesses, in the Smart tariff, on the contrary, in 75% of cases users pay extra for exceeding the limit.

- The average revenue for the ultra tariff greatly exceeds the revenue for the smart tariff, but at the same time, the standard deviation and variance for the smart tariff are higher, which indicates a high variability of the random variable.

- There is more data on the smart tariff than on the ultra tariff, which is logical, since the ultra tariff is more expensive and therefore less popular.

- Users of the Ultra tariff, on average, write SMS more often, use the mobile Internet more and talk on the phone more than users of the Smart tariff, this is logical, since Ultra tariff limits allow you to do this simply.

- As a result of this study, we confirmed the hypothesis that the average revenue for Smart and Ultra tariffs differ, and also refuted the hypothesis that the average revenue in Moscow and regions differ, as it turned out, there is practically no difference.

- I think that it makes sense to focus on advertising Ultra tariff, offering high Internet limits and the ability to talk almost without restrictions, without being afraid to exceed the limits.
